In [12]:
from langchain.retrievers import BM25Retriever
from sentence_transformers import SentenceTransformer, util
from sentence_transformers import models, evaluation
import os
import pandas as pd
import warnings 
warnings.filterwarnings("ignore")
pretrain_model_path = "/Users/zyl/Desktop/LLM/pretrain_models/all-MiniLM-L6-v2"
model = SentenceTransformer(pretrain_model_path)

In [6]:
label123 = ['功能号维度交易成功率低', '高压报警', '频率过高报警', '存储重要端口down', '响应率低', '消息队列异常',
       '中间件连接池满', '空调关机报警', '输入电压异常', 'WAS服务数据源连接池使用率高', '进程个数异常',
       '中间件jvm内存溢出', '数据库表缓存过高', 'VPN应用使用人数多', '功能号维度交易量无推送', '天窗状态告警',
       '主机连接和电源状态', '数据库大事务存在', '容器内存率高', 'web后门存在', '数据库IO等待',
       'LED状态为on', '批量作业执行出错', '数据库死锁存在', '临时地址使用率过高', '数据库集群文件系统状态异',
       '存储设备未挂载', '通道湿度上限', '送风湿度上限告警', '*数据库服务异常', '服务器datastore异常',
       '风扇异常告警', '传感器通讯失败', '进程个数大于1', '数据库共享文件系统使用率', '中间件线程hang住',
       '前置机请求积压', '光交重要端口down', '数据库租户表空间使用率高', 'Redis服务连接数过多',
       '数据库主备状态异常', '端口流出带宽利用率高', '通风设备故障', 'Oracle数据库应用总集群活动会话数高',
       '存储重要端口利用率高', '数据库日志同步等待', 'Mysql数据库集群节点异常', 'os宕机机', '业务性能下降',
       '应用中间件线程池使用率高', '磁盘IO繁忙', '服务器电池异常', '日间总分核对失败', '交易量低',
       '功能号维度交易响应变慢', '光线路保护倒换', '队列快照文件', '功耗异常告警', '业务日志错误',
       '功能号维度交易量突增', 'HMC仲裁失败', '消息积压', '数据库表空间无法扩展', '中间件jvm使用率高',
       '同步站内信失败', '应用进程异常', '数据库并行度未开', '数据库连接会话数异常',
       'GoldenGateReplicat进程未运行', '功能号维度交易量持续低', 'Oracle数据库ASMDATA/使用率高',
       '存在故障盘', '工作队列使用率过高', '中间件gc发生异常', '气体浓度过高报警',
       '开关合闸', '信号失步', '数据异常', '流量利用率高', 'SDATA数据异常', '日终任务执行出错', 
       '数据存储占用率高', 'UPS电源异常', '数据库实例不可用', '分行维度交易成功率低', '数据库事务日志编号使用率高',
       '数据库集群IP状态异常', '连接状态异常', '登录耗时告警', '数据库租户memstore使用百分比超限',
       '数据库定时任务失败', '数据库慢sql多次执行', '通信失败', '数据库不循环序列使用率超限', '数据库发生死锁',
       '断路器合闸报警', 'NTP服务器心跳异常', 'ActiveJobCompletedwithExitStatus',
       '数据库事务日志编号使用率', '数据库归档使用率高', '数据库打开游标数高', 'FC端口异常告警', '业务外联异常',
       '无法连接到存储设备', '分行维度交易量无推送', '漏水告警', '内存异常', '分行维度交易响应变慢', '数据库操作错误',
       '*带宽利用率高', '数据库日志重要报错', '数据库等待事件异常', '慢接口平均时间大于阈值的次数超过阈值',
       'Oracle数据库同步进程未启动', '数据库服务异常', '交易码维度交易量持续低', '定时任务进程未关闭',
       '精密空调通信中断', '集群节点健康状况异常', '交易码维度交易成功率低', '通信质量告警', '容器内存率高',
       'os集群文件系统状态异常', '数据库登录失败次数过多', '主备线路故障', '数据库实例离线', '数据库序列无缓存',
       '数据库状态异常', '数据库重启', '主备数据库延迟', '分区状态异常', '机房ups异常', 'os宕机',
       'Ping不通', '阵列卡异常告警', '通道状态异常', '设备通讯异常', '响应时间超阀值', '设备cpu故障',
       '内存使用率高', '交易响应变慢', '应用中间件连接池使用率高', '交易量异常&成功率低', '主交流开关状态异常',
       'os虚拟机漂移失败', 'OBserver进程不存在', '应用中间件服务异常', '通道总数高', '作业状态异常',
       '定时任务失败', '手动位报警', '数据库锁等待存在', '缓存主备延迟', '数据库进程数使用率高',
       '服务器物理内存使用率高', '数据库全表扫描', 'os系统日志硬件关键错误', 'NGINX日志文件大小超过阈值',
       '文件系统使用率高', '物理端口链路状态异常', '交易成功率低', '物理磁盘异常告警', '系统数据异常', '开关分位报警',
       'CPU使用率异常', '数据库进程限制过高', '表空间使用率高', '消息深度异常', '死信队列未定义',
       '中间件连接池使用率高', '网络传输延迟高', '时效性监控报错', '服务IP不通', 'Oracle数据库实例重启',
       '数据库会话阻塞', 'Tomcat连接池使用率高', '数据库集群文件系统状态异常', '响应时间和交易量异常', '容器重启',
       'Ping中断', '链路状态异常', '数据库内存溢出', 'OB日志告警', 'Microservice线程池使用率高',
       '服务器宕机', '接口状态异常', 'os文件系统使用率高', '业务连续性压力测试',
       '数据库主备同步进程未启动', '采集设备通讯失败', '缓存重启', '内存溢出', 'DGCHECK故障',
       '数据库数据介质异常', 'pool成员不可用', '数据库共享文件系统使用率高', '*应用中间件线程池使用率高',
       'Ping超时', '数据库排序SQL缓存过高', 'DataGuard状态异常', '邮件数量超过500', 'Fex离线',
       '通道温度异常', '请求长度不匹配', '分行维度交易量突增', '数据库索引不可用', '**网络传输延迟高',
       '空调压缩机高压报警', '设备down', '应用状态down', '缓存阻塞会话', '流量控制生效', '通讯异常',
       '缓存无法连接', '业务发送方长交易', 'OBProxy不可用', '队列管理器状态异常', '数据库对象失效', '导出失败',
       '数据库主备延迟', '进程未运行', '数据库集群IP状态异常', '日志异常告警', 'WideIP状态变化',
       '*前置机异常', 'Thestatusofthenon-virtualneighborhaschanged.', '分合状态异常',
       '服务器重启', '缓存qps高', '线路中断', 'NGINX连接数达上限', '应用sql执行错误', '数据库监听状态异常',
       '响应时间高', 'OSPF邻接状态异常', '弹簧未储能报警', '服务器链路异常', '开关断开报警', '业务外联异常',
       '数据库回话等待', '水冷空调报警', '自动位报警', '进程个数小于阈值',
       'OutFlowUsageofaHostNicExceedstheThreshold', 'NTP时钟偏离',
       '交易码维度交易量无推送', '应用拨测检测失败', 'cpu使用率高', '网络CRC错误严重', 'os虚拟机漂移',
       '库在线状态异常', '数据库索引竞争存在', 'Clusterware在主代理主机上有问题', 'Redis内存使用率高',
       '日志备份延迟', '缓存内存使用高', 'Microservice空闲JVM使用率高', '存储复制端口利用率高',
       '其他异常告警', '电源状态异常', '业务文件未更新', 'os文件系统inode使用率高', 'WAS服务线程池使用率高',
       '数据库后台进程异常', '数据库集群文件系统状态', '响应时间突增', '*缓存主备状态异常', '中间件jvm使用率高',
       '网络线路异常', '中间件线程池使用率高', '交易码维度交易量突增', '数据库连接会话数异常', '应用状态异常',
       '供油泵运行报警', '硬件服务提供商问题', 'Monitor状态异常', '网络传输丢包率高', '数据库发生重启',
       '数据库临时表空间状态异常', '前置机请求积压', '交易量无推送', '数据库异常等待', '磁盘IO延迟高',
       '成功率低', '服务器cpu故障', '市电停电报警', '数据库集群节点异常', '水冷系统需减机', '空开断开',
       '设备内存故障', '数据异常作业未完成', '交易量突增', '业务接收方长交易', 'HBA状态异常',
       'Redis节点同步状态异常', '服务器监控状态下线', '设备状态异常', '监控代理掉线', 'os文件系统inode使用率',
       '数据库DataGuard状态异常', '中间件javacore诊断文件', '数据库表空间使用率高', '缓存内存碎片化',
       '数据库连接超限', 'os系统日志软件关键错误', '网卡状态down', 'Redis服务异常',
       'GlobalCacheBlocksLost', 'CNA和VRM之间的心跳通信中断',
       'GoldenGateExtractProcessisnotRunning', '交易量持续低', '进程数使用率过高',
       '主备同步状态异常', '消息日志错误', 'Oracle数据库实例Incident', '前置机异常',
       '数据库不循环序列使用率超', '网络重要端口down', '数据库连接失败', '服务器物理cpu使用率高', '阈值设定异常',
       '应用中间件线程hang住', '数据库索引不规范', '集群资源异常', '数据库监听状态异常', 'OB集群冻结检测失败',
       '获取数据库集群信息失败', '端口异常告警', '接口物理状态为down', '应用中间件连接池使用率高',
       '数据库日志重要报错', '连接池使用率过高', 'BGP邻接关系异常', '服务器物理内存故障', '分行维度交易量持续低',
       '端口流入带宽利用率高', '应用中间件javacore诊断文件生成', 'exporter数量不足', '数据库日志损坏',
       'agent服务不可用', '交易量高于阈值&成功率低于阈值', '服务器光纤卡吞吐量达', '存储响应时间超阈值',
       '温度过高报警', '应用状态DOWN', '数据库表空间状态异常', '带宽利用率高', '硬件操作错误', '中间件服务异常',
       'GlobalCacheAverageCRGetTime过高', 'os系统磁盘错误', '超最大连接数',
       '用户进程句柄数的百分比使用率高', '数据库缓存命中率低', '存储重要端口down', '带宽使用超过阈值',
       '数据库表的degree不为1', '数据中断', '主备设备故障', '交易错误码统计结果大', 'WPA入库失败',
       '主机网络不可达', '进程不存在', '数据库内存组件异常', '交易量异常&响应率低', '开关合位报警',
       'Web服务状态异常', '缓存主备状态异常', 'Ping成功率低', 'Tomcat线程池使用率高', '业务文件不存在',
       'Redis服务QPS高', 'vSphere主机状态异常', '网络冗余丢失告警', 'Oracle高可用性服务无法联系',
       '硬盘操作错误', '应用sql执行慢', '缓存进程掉线', '交易码维度交易响应变慢']
len(label123)

379

In [11]:
sentences = ['内存使用率高', 'Ping不通', 'Ping成功率低', '线路中断', '应用状态DOWN', '作业状态异常',
       '传感器通讯失败', 'CPU使用率异常', 'pool成员不可用', '市电停电报警', '端口异常告警', '数据库实例不可用',
       '响应率低', '应用状态down', '进程个数异常', 'Ping中断', '功耗异常告警', '设备状态异常',
       'Oracle数据库应用总集群活动会话数高', '网卡状态down', '电源状态异常', '风扇异常告警', '表空间使用率高',
       'Redis服务异常', '通讯异常', '主备数据库延迟', '存储响应时间超阈值', '温度过高报警', '网络冗余丢失告警',
       '服务器重启', '设备通讯异常', 'NTP时钟偏离', '流量利用率高', '日终任务执行出错', 'UPS电源异常',
       '数据库会话阻塞', '通道湿度上限', '主备线路故障', '网络线路异常', '业务日志错误', '文件系统使用率高',
       '用户进程句柄数的百分比使用率高', '接口状态异常', '数据库状态异常', 'Oracle数据库实例重启', '通信质量告警',
       'DGCHECK故障', '链路状态异常', 'DataGuard状态异常', '邮件数量超过500', '光线路保护倒换',
       'BGP邻接关系异常', '接口物理状态为down', '数据库发生死锁', '时效性监控报错', '系统数据异常',
       '存储设备未挂载', 'Redis内存使用率高', 'Tomcat线程池使用率高', 'Microservice线程池使用率高',
       '交易量低', 'agent服务不可用', '数据库索引不可用', '响应时间超阀值', 'Ping超时',
       'WideIP状态变化', 'Tomcat连接池使用率高', 'NGINX连接数达上限', '主机连接和电源状态',
       '数据存储占用率高', '信号失步', '存在故障盘', '数据库锁等待存在', '数据库实例离线', 'OSPF邻接状态异常',
       '内存异常', '主备同步状态异常', '通信失败', 'Redis服务QPS高', '消息队列异常', 'OB集群冻结检测失败',
       '数据库回话等待', 'WAS服务数据源连接池使用率高', 'WAS服务线程池使用率高',
       'Thestatusofthenon-virtualneighborhaschanged.', 'LED状态为on',
       '数据库服务异常', '数据库集群节点异常', '输入电压异常', '内存溢出', 'NGINX日志文件大小超过阈值',
       '物理磁盘异常告警', '硬盘操作错误', 'Fex离线', '气体浓度过高报警',
       'CNA和VRM之间的心跳通信中断', 'Oracle数据库同步进程未启动', '响应时间高', '库在线状态异常', '高压报警',
       '弹簧未储能报警', '开关分位报警', '开关合位报警', '自动位报警', '手动位报警', '供油泵运行报警',
       '其他异常告警', '数据库发生重启', 'Oracle数据库ASMDATA/使用率高', '水冷系统需减机', '开关合闸',
       '进程未运行', 'FC端口异常告警', '开关断开报警', 'Microservice空闲JVM使用率高', '日志异常告警',
       'vSphere主机状态异常', 'Web服务状态异常',
       'GoldenGateExtractProcessisnotRunning', 'OB日志告警', '断路器合闸报警',
       '水冷空调报警', '网络传输延迟高', '数据异常', '数据中断', '空调关机报警',
       'GlobalCacheBlocksLost', '同步站内信失败', '日间总分核对失败', '进程个数大于1',
       'OBProxy不可用', '数据库不循环序列使用率超限',
       'Oracle数据库实例Incident (ORA 800)检测到严重错误', 'SDATA数据异常', '基金昨日收益为0',
       '响应时间突增', '数据异常作业未完成', '通风设备故障', '频率过高报警',
       'OutFlowUsageofaHostNicExceedstheThreshold', '登录耗时告警',
       '数据库进程数使用率高', '数据库连接超限', '业务性能下降', '业务连续性压力测试', '带宽使用超过阈值',
       '服务器监控状态下线', '无法连接到存储设备', '送风湿度上限告警', '通道温度异常', '交易量异常&成功率低',
       '空调压缩机高压报警', '数据库DataGuard状态异常', '应用状态异常', '硬件服务提供商问题', '成功率低',
       '主机网络不可达', '集群节点健康状况异常', '物理端口链路状态异常', 'Redis节点同步状态异常',
       '获取数据库集群信息失败', '数据库连接失败', 'exporter数量不足', '数据库表的degree不为1',
       'Oracle高可用性服务无法联系', '数据库日志损坏', '数据库租户memstore使用百分比超限',
       '交易量异常&响应率低', '阈值设定异常', '导数失败', '日志备份延迟', '硬件操作错误', 'web后门存在',
       '监控代理掉线', '批量作业执行出错', '数据库等待事件异常', 'HBA状态异常', '数据库操作错误',
       '数据库进程限制过高', '连接状态异常', 'GoldenGateReplicat进程未运行', 'OBserver进程不存在',
       'Redis服务连接数过多', '精密空调通信中断', '进程不存在', '空开断开', '数据库重启', '进程数使用率过高',
       '漏水告警', '阵列卡异常告警', '连接池使用率过高', 'VPN应用使用人数多', '天窗状态告警',
       'Mysql数据库集群节点异常', '响应时间和交易量异常', '分区状态异常', 'NTP服务器心跳异常',
       '临时地址使用率过高', '交易量高于阈值&成功率低于阈值', 'ActiveJobCompletedwithExitStatus',
       'Clusterware在主代理主机上有问题', '进程个数小于阈值', '服务IP不通', '主交流开关状态异常',
       '分合状态异常', '慢接口平均时间大于阈值的次数超过阈值', '工作队列使用率过高', 'WPA入库失败', 'HMC仲裁失败',
       'Snap Shot Monitor状态异常', 'GlobalCacheAverageCRGetTime过高']
print(len(sentences), "token---", len(str(sentences)))

212 token--- 2800


In [14]:
from tqdm import tqdm
embs = []
for i, label in tqdm(enumerate(sentences)):
    emb = model.encode(label, convert_to_tensor=True)
    embs.append(emb)

212it [00:01, 110.20it/s]


In [146]:
import pandas as pd
alarm_path = "/Users/zyl/Desktop/东亚银行/dy-temp835.csv"
alarm = pd.read_csv(alarm_path, usecols=['gpt'])

## BM25

In [37]:
retriever = BM25Retriever.from_texts(sentences)
result = retriever.get_relevant_documents("流量超阈值")
result

[Document(page_content='GlobalCacheAverageCRGetTime过高'),
 Document(page_content='Redis服务QPS高'),
 Document(page_content='主备同步状态异常'),
 Document(page_content='内存异常')]

## Sentence Transformer

In [23]:
gpt_labels = alarm.gpt.values.tolist()
gpt_embs = []
sim_matrix = []
labels = []

for gpt_label in tqdm(gpt_labels):
    gpt_emb = model.encode(gpt_label, convert_to_tensor=True)
    gpt_embs.append(gpt_emb)
for gpt_emb in tqdm(gpt_embs):
    sim_list = []
    for emb in embs:
        cos_sim = util.cos_sim(gpt_emb, emb)
        sim_list.append(cos_sim.item())
    sim_matrix.append(sim_list)
print(len(embs), len(gpt_embs))
for i, sim_list in tqdm(enumerate(sim_matrix)):
    ind = sim_list.index(max(sim_list))
    labels.append(sentences[ind])
print(len(labels))

100%|███████████████████████████████████████████████████████████████████████████| 835/835 [00:07<00:00, 117.66it/s]


In [38]:
# alarm['label'] = labels
alarm.head(10)

,gpt,label
0,CPU 使用率高,CPU使用率异常
1,CPU 使用率高,CPU使用率异常
2,带宽超限,响应率低
3,带宽超限,响应率低
4,流量超阈值,服务器重启
5,流量超阈值,服务器重启
6,内存使用率高,内存使用率高
7,内存使用率高,内存使用率高
8,进程数低于阈值,传感器通讯失败
9,设备无响应,服务器重启


## embedding+faiss

In [3]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain.embeddings import CacheBackedEmbeddings, HuggingFaceEmbeddings
from langchain.storage import LocalFileStore
from langchain.retrievers import  BM25Retriever, EnsembleRetriever

In [6]:
from langchain.vectorstores import FAISS
import pandas as pd
import os
import getpass

# sk-i2tPshWiPNgoeTooKILYVGhlQi5BSQAc16xpjOzyJpGyVU0C
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
embedding_path = "/Users/zyl/Desktop/LLM/pretrain_models/"

OpenAI API Key: ········


#### 1. openaiembedding

In [4]:
loader = TextLoader('/Users/zyl/Desktop/东亚银行/标签库.txt')

documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=10, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

Created a chunk of size 20, which is longer than the specified 10
Created a chunk of size 15, which is longer than the specified 10
Created a chunk of size 13, which is longer than the specified 10
Created a chunk of size 13, which is longer than the specified 10
Created a chunk of size 11, which is longer than the specified 10
Created a chunk of size 11, which is longer than the specified 10
Created a chunk of size 13, which is longer than the specified 10
Created a chunk of size 19, which is longer than the specified 10
Created a chunk of size 13, which is longer than the specified 10
Created a chunk of size 11, which is longer than the specified 10
Created a chunk of size 11, which is longer than the specified 10
Created a chunk of size 15, which is longer than the specified 10
Created a chunk of size 12, which is longer than the specified 10
Created a chunk of size 44, which is longer than the specified 10
Created a chunk of size 15, which is longer than the specified 10
Created a 

In [56]:
for i, d in enumerate(docs):
    print("第{}条：".format(i), d.page_content)
    if i > 10:
        break

第0条： 内存使用率高
第1条： Ping不通
第2条： Ping成功率低
第3条： 线路中断
第4条： 应用状态DOWN
第5条： 作业状态异常
第6条： 传感器通讯失败
第7条： CPU使用率异常
第8条： pool成员不可用
第9条： 市电停电报警
第10条： 端口异常告警
第11条： 数据库实例不可用


In [7]:
embeddings = OpenAIEmbeddings()

In [5]:
db = FAISS.from_documents(docs, embeddings)

/Users/zyl/miniforge3/envs/tensorflow2_8/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. Use langchain_openai.OpenAIEmbeddings instead.
  warn_deprecated(
Retrying langchain_community.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors..


RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.

In [64]:
openai_labels = []
for label in tqdm(gpt_labels):
    doc = db.similarity_search(label)
    print("gpt_label, openai_label is :", label, doc[0].page_content)
    # openai_labels.append(doc[0].page_content)
# alarm['openai_labels'] = openai_labels
# alarm.head(10)
"""
gpt_label, openai_label is : CPU 使用率高   CPU使用率异常
gpt_label, openai_label is : 带宽超限       带宽使用超过阈值
gpt_label, openai_label is : 流量超阈值      带宽使用超过阈值
gpt_label, openai_label is : 内存使用率高    内存使用率高
gpt_label, openai_label is : 进程数低于阈值  进程个数小于阈值
gpt_label, openai_label is : 设备无响应      设备通讯异常
gpt_label, openai_label is : CPU 负载高     CPU使用率异常
gpt_label, openai_label is : 进程数过多      进程数使用率过高
gpt_label, openai_label is : 备份失败        通信失败
gpt_label, openai_label is : 数据库表空间使用率高 表空间使用率高
gpt_label, openai_label is : 磁盘空间低      硬盘操作错误
gpt_label, openai_label is : 设备无法访问    无法连接到存储设备
gpt_label, openai_label is : 作业异常        作业状态异常
gpt_label, openai_label is : 丢包率高        数据存储占用率高
gpt_label, openai_label is : 接口关闭        接口状态异常
gpt_label, openai_label is : 流量超阈值      带宽使用超过阈值
gpt_label, openai_label is : 带宽超限        带宽使用超过阈值
"""

  0%|                                                                              | 1/835 [00:00<09:59,  1.39it/s]

gpt_label, openai_label is : CPU 使用率高 CPU使用率异常


  0%|▏                                                                             | 2/835 [00:01<07:36,  1.82it/s]

gpt_label, openai_label is : CPU 使用率高 CPU使用率异常


Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-embedding-ada-002 in organization org-DCKsnDzKQrua8NQgl17yrHGl on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
  0%|▎                                                                             | 3/835 [00:05<34:37,  2.50s/it]

...

gpt_label, openai_label is : 带宽超限 带宽使用超过阈值


Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-embedding-ada-002 in organization org-DCKsnDzKQrua8NQgl17yrHGl on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-embedding-ada-002 in organization org-DCKsnDzKQrua8NQgl17yrHGl on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langcha

gpt_label, openai_label is : 带宽超限 带宽使用超过阈值


Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-embedding-ada-002 in organization org-DCKsnDzKQrua8NQgl17yrHGl on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-embedding-ada-002 in organization org-DCKsnDzKQrua8NQgl17yrHGl on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
  3%|██▎        

KeyboardInterrupt: 

## 2. m3e

In [191]:
from sentence_transformers import SentenceTransformer
model_name = "/Users/zyl/Desktop/LLM/pretrain_models/m3e-base"
m3e_model = SentenceTransformer(model_name)
# m3e_embs = model.encode(sentences)
# m3e_embs.shape

(212, 768)

In [88]:
gpt_labels = alarm.gpt.values.tolist()
gpt_embs = []
sim_matrix = []
labels = []

for gpt_label in tqdm(gpt_labels):
    gpt_emb = m3e_model.encode(gpt_label, convert_to_tensor=True)
    gpt_embs.append(gpt_emb)

In [90]:
for gpt_emb in tqdm(gpt_embs):
    sim_list = []
    for emb in m3e_embs:
        cos_sim = util.cos_sim(gpt_emb, emb)
        sim_list.append(cos_sim.item())
    sim_matrix.append(sim_list)
print(len(m3e_embs), len(gpt_embs))
for i, sim_list in tqdm(enumerate(sim_matrix)):
    ind = sim_list.index(max(sim_list))
    labels.append(sentences[ind])
print(len(labels))

100%|███████████████████████████████████████████████████████████████████████████| 835/835 [00:04<00:00, 198.37it/s]


212 835


835it [00:00, 375817.56it/s]

835


In [92]:
# alarm['m3e_label'] = labels
alarm.head(30)

,gpt,label,m3e_label
0,CPU 使用率高,CPU使用率异常,CPU使用率异常
1,CPU 使用率高,CPU使用率异常,CPU使用率异常
2,带宽超限,响应率低,带宽使用超过阈值
3,带宽超限,响应率低,带宽使用超过阈值
4,流量超阈值,服务器重启,带宽使用超过阈值
5,流量超阈值,服务器重启,带宽使用超过阈值
6,内存使用率高,内存使用率高,内存使用率高
7,内存使用率高,内存使用率高,内存使用率高
8,进程数低于阈值,传感器通讯失败,进程个数小于阈值
9,设备无响应,服务器重启,设备状态异常


#### 3. text2vec-large

In [51]:
# fout = open('/Users/zyl/Desktop/东亚银行/标签库.txt', 'w')
# for label in sentences:
#     fout.write(label+'\n')
#     fout.write('\n')
#     fout.flush()
# fout.close()

In [93]:
alarm = alarm.rename(columns={'label':'sentence_transformer_label'})
alarm.head()

,gpt,sentence_transformer_label,m3e_label
0,CPU 使用率高,CPU使用率异常,CPU使用率异常
1,CPU 使用率高,CPU使用率异常,CPU使用率异常
2,带宽超限,响应率低,带宽使用超过阈值
3,带宽超限,响应率低,带宽使用超过阈值
4,流量超阈值,服务器重启,带宽使用超过阈值


In [94]:
alarm.to_csv("/Users/zyl/Desktop/东亚银行/标签映射到已有库.csv", index=False)

## 下载bge预训练模型

In [96]:
from transformers import AutoTokenizer, AutoModel
import torch
tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-base-zh-v1.5')
model = AutoModel.from_pretrained('BAAI/bge-base-zh-v1.5')
save_path = "/Users/zyl/Desktop/LLM/pretrain_models/bge-base-zh-v1.5/"

tokenizer.save_pretrained(save_path)
model.save_pretrained(save_path)

## bge-base-zh

In [105]:
from sentence_transformers import SentenceTransformer

In [192]:
# 加载bge模型
model_path = "/Users/zyl/Desktop/LLM/pretrain_models/bge-base-zh-v1.5"
bge_model = SentenceTransformer(model_path)

No sentence-transformers model found with name /Users/zyl/Desktop/LLM/pretrain_models/bge-base-zh-v1.5. Creating a new one with MEAN pooling.


In [187]:
# 加载文件
nb_path = "/Users/zyl/Desktop/打标签/辽宁农信模版告警.csv"
# dy_path = "/Users/zyl/Desktop/东亚银行/dy-temp835.csv"
df = pd.read_csv(nb_path)
df.head(2)

,content,EventId,gpt_label,label_without（没范围）,Unnamed: 4,label_with（包含范围）,label_with(正确率70%),bge_label
0,windows设备宕机,0,设备宕机,主机不可用,NaN,主机不可用,1,主机网络不可达
1,华为交换机宕机,1,设备宕机,主机不可用,NaN,主机不可用,1,主机网络不可达


(768,) torch.Size([768])
{-0.09202159: -0.9736336469650269,
 -0.09098431: -1.2923485040664673,
 -0.08539493: -0.1390996128320694,
 -0.08276554: -1.726261854171753,
 -0.08224665: -0.45424970984458923,
 -0.080686465: -1.0303500890731812,
 -0.07954146: 0.380729615688324,
 -0.07917325: -0.9921663999557495,
 -0.07788336: -0.02524111047387123,
 -0.07359229: -1.4387128353118896,
 -0.073058195: -0.9597848057746887,
 -0.06925815: -1.013558030128479,
 -0.066257745: -1.2376374006271362,
 -0.065388255: 0.2854411005973816,
 -0.064997785: -1.0525201559066772,
 -0.064538956: 0.2524532675743103,
 -0.063648365: -1.0173975229263306,
 -0.063141204: 0.31023502349853516,
 -0.063043214: -0.4324498176574707,
 -0.062278546: -1.5814275741577148,
 -0.061612483: -0.24800491333007812,
 -0.060318545: -0.15330271422863007,
 -0.059386697: -0.1529470980167389,
 -0.05887122: -0.475502073764801,
 -0.058427297: 0.14419804513454437,
 -0.058304902: -0.8470115661621094,
 -0.057966318: -0.6769821047782898,
 -0.057911206: 0.

In [188]:
def bge_calculate(df, sentences, label1, model):
    embeddings_1 = model.encode(sentences, normalize_embeddings=True)
    label1_ = df[label1].tolist()
    map_labels = []
    for label in tqdm(label1_):
        embeddings_2 = model.encode(label, normalize_embeddings=True)
        similarity = embeddings_1 @ embeddings_2.T
        ind_ = similarity.flatten().argmax()
        map_labels.append(sentences[ind_])
    df['bge_label'] = map_labels
    return df

def add_T(df, label1, label2):
    label1_ = df.label1.tolist()
    label2_ = df.label2.tolist()
    T = []
    for i in range(len(label1_)):
        if label1_[i] == label2_[i]:
            T.append(1)
        else:
            T.append(0)
    df['compare'] = T
    return df

def save_df(df, save_path):
    df.to_csv(save_path, index=False)
    

In [190]:
# new_df = bge_calculate(df, sentences, label1="label_without（没范围）", model=bge_model)
# new_df.head()
# save_df(df, save_path=nb_path)
# add_T(new_df, label1, label2)

In [159]:
# sentences_1 = sentences
# embeddings_1 = model.encode(sentences_1, normalize_embeddings=True)
# label_without = df_nb.label_without.tolist()
# map_labels = []
# for label in tqdm(label_without):
#     embeddings_2 = model.encode(label, normalize_embeddings=True)
#     similarity = embeddings_1 @ embeddings_2.T
#     ind_ = similarity.flatten().argmax()
#     map_labels.append(sentences[ind_])
# df_nb['bge_label'] = map_labels

# print(similarity.flatten(), similarity.flatten().argmax())

100%|████████████████████████████████████████████████████████████████████████████| 835/835 [01:20<00:00, 10.40it/s]


In [162]:
# m3e_label = df_nb.gpt_m3e_label.tolist()
# bge_label = df_nb.bge_label.tolist()
# T = []
# for i in range(len(m3e_label)):
#     if m3e_label[i] == bge_label[i]:
#         T.append(1)
#     else:
#         T.append(0)
# df_nb['m3e&bge'] = T

In [165]:
# dy_path = "/Users/zyl/Desktop/东亚银行/dy-temp835.csv"
# df_nb.to_csv(dy_path, index=False)

In [163]:
df_nb.head(50)

,content,EventId,gpt_label,level,gpt_m3e_label,gpt_bge_label,gpt_m3e&bge,label_without,bge_label,m3e&bge
0,128.18.176.206Windows CPU使用率:88.44 %CPU 使用率大于8...,0,CPU 使用率高,4.0,CPU使用率异常,内存使用率高,0,CPU使用率异常,CPU使用率异常,1
1,"128.20.65.52CPU使用率:89.04 %CPU 使用率大于80%, 当前值为(8...",1,CPU 使用率高,4.0,CPU使用率异常,内存使用率高,0,CPU使用率异常,CPU使用率异常,1
2,带宽超出接口GigabitEthernet0/3-Gi0/3配置的限制。当前流入流量是226...,2,带宽超限,4.0,带宽使用超过阈值,带宽使用超过阈值,1,接口流量利用率高,流量利用率高,0
3,带宽超出接口Vlan1086-Vlan1086配置的限制。当前流入流量是1.072 Gbps...,3,带宽超限,4.0,带宽使用超过阈值,带宽使用超过阈值,1,接口流量利用率高,流量利用率高,0
4,接口'GigabitEthernet0/0/2.103-MSTP_CT_HCBM_M1288...,4,流量超阈值,4.0,带宽使用超过阈值,带宽使用超过阈值,1,流量利用率高,流量利用率高,0
5,接口'ethernet1/1-CUC-220.248.51.65-78'违反阈值。当前流入流...,5,流量超阈值,4.0,带宽使用超过阈值,带宽使用超过阈值,1,阈值流量违反,带宽使用超过阈值,1
6,128.20.68.11Windows内存使用率:83.2 %内存使用率大于80%，当前值为...,6,内存使用率高,4.0,内存使用率高,内存使用率高,1,内存使用率高,内存使用率高,1
7,"128.18.163.153内存使用率:83 %内存使用大于80%, 当前值为(83 %)",7,内存使用率高,4.0,内存使用率高,内存使用率高,1,内存使用率高,内存使用率高,1
8,128.18.73.45Number of running processes:9PLBDP...,8,进程数低于阈值,2.0,进程个数小于阈值,进程个数小于阈值,1,进程数使用率过高,进程数使用率过高,0
9,设备没有响应：设备可能已关机或者设备太忙,9,设备无响应,4.0,设备状态异常,设备状态异常,1,设备状态异常,设备状态异常,1


## embedding & sim

In [210]:
# test embedding
from pprint import pprint
from sentence_transformers import util

In [273]:
sentence = "CPU使用率高"
sentence1 = "CPU使用率异常"   # ----> m3e
sentence2 = "内存使用率高"    # ----> bge

# sentence = "磁盘空间低"
# sentence1 = "数据存储占用率高"   # ----> m3e
# sentence2 = "数据存储占用率高"   # ----> bge

# sentence = "带宽超限"
# sentence1 = "带宽使用超过阈值"   # ----> m3e
# sentence2 = "流量超阈值"   # ----> bge

In [276]:
# 1. 两种嵌入方法对比
bge_emb = bge_model.encode(sentence, normalize_embeddings=True)
m3e_emb = m3e_model.encode(sentence, convert_to_tensor=True)
print(bge_emb.shape, m3e_emb.shape)
# pprint(dict(zip(bge_emb, [emb.item() for emb in m3e_emb])))
print(bge_emb)

(768,) torch.Size([768])
[-8.07060394e-03 -2.40585934e-02  6.85756058e-02 -3.43218772e-03
 -8.89241695e-03 -1.56662781e-02  2.23670471e-02 -1.11223683e-02
  7.89475255e-03  1.60926245e-02 -4.59752381e-02  1.72967948e-02
  4.06720527e-02 -4.97396924e-02 -4.60541202e-03  1.85309611e-02
 -4.03384715e-02 -1.71454903e-02 -1.75561458e-02  8.40970688e-03
  4.27505672e-02  3.85058336e-02 -1.53017882e-02 -2.60895919e-02
  8.95911828e-03  2.52698157e-02  8.99586361e-03  3.92565429e-02
  7.67715648e-02  9.85879358e-03  5.39837703e-02  1.82226766e-02
 -4.26422898e-03 -3.94298248e-02 -5.34725115e-02  2.91242935e-02
  1.73616502e-03 -1.81612801e-02 -6.73725316e-03  4.55870852e-02
 -2.87112631e-02  1.24694230e-02  6.05683848e-02  1.25730701e-03
  1.06355920e-02  8.10504984e-03  2.70831329e-03 -6.31412044e-02
 -4.86819707e-02  1.38273900e-02  8.27083271e-03  2.91575044e-01
 -2.76675895e-02 -4.05083075e-02 -1.24552399e-02  4.68856655e-02
  9.92361549e-03  1.16662458e-02  6.49075061e-02 -3.28880665e-03


In [275]:
# 同一种嵌入方法对比
bge_emb1 = bge_model.encode(sentence, normalize_embeddings=True)
bge_emb2 = bge_model.encode(sentence1, normalize_embeddings=True)
pprint(dict(zip(bge_emb1, bge_emb2)))

{-0.09202159: -0.0830294,
 -0.09098431: -0.07534003,
 -0.08539493: -0.03452983,
 -0.08276554: -0.080405526,
 -0.08224665: -0.07168121,
 -0.080686465: -0.072711565,
 -0.07954146: -0.056492653,
 -0.07917325: -0.05996134,
 -0.07788336: -0.05150594,
 -0.07359229: -0.070762485,
 -0.073058195: -0.058754206,
 -0.06925815: -0.058171738,
 -0.066257745: -0.05399446,
 -0.065388255: -0.043690972,
 -0.064997785: -0.07167244,
 -0.064538956: -0.050972473,
 -0.063648365: -0.04809918,
 -0.063141204: -0.034786426,
 -0.063043214: -0.06818667,
 -0.062278546: -0.02763573,
 -0.061612483: -0.020360585,
 -0.060318545: -0.047053702,
 -0.059386697: -0.028676348,
 -0.05887122: -0.021782653,
 -0.058427297: -0.043463062,
 -0.058304902: -0.041409582,
 -0.057966318: -0.05942841,
 -0.057911206: -0.06820434,
 -0.05672225: -0.062310174,
 -0.056082282: -0.028324286,
 -0.05584649: -0.012107629,
 -0.055450473: 0.0061876536,
 -0.054856196: -0.06617432,
 -0.054511964: -0.043053985,
 -0.05379977: -0.077149116,
 -0.05348132: 

In [259]:
# 2. m3e计算相似度
m3e_emb1 = m3e_model.encode(sentence1, convert_to_tensor=True)
m3e_emb2 = m3e_model.encode(sentence2, convert_to_tensor=True)
print('*'*50)
print('                    m3e-base')
print('*'*50)
# sentence 和 sentence1 相似度
sen_sen1 = util.cos_sim(m3e_emb, m3e_emb1)
print("{} 和 {} 相似度: {}".format(sentence, sentence1, sen_sen1.item()))

# sentence 和 sentence2 相似度
sen_sen2 = util.cos_sim(m3e_emb, m3e_emb2)
print("{} 和 {} 相似度: {}".format(sentence, sentence2, sen_sen2.item()))

**************************************************
                    m3e-base
**************************************************
带宽超限 和 带宽使用超过阈值 相似度: 0.9190897941589355
带宽超限 和 流量超阈值 相似度: 0.8416625261306763


In [260]:
# 3. bge计算相似度
bge_emb1 = bge_model.encode(sentence1, convert_to_tensor=True)
bge_emb2 = bge_model.encode(sentence2, convert_to_tensor=True)
print('*'*50)
print('                 bge-base-zh-v1')
print('*'*50)
# sentence 和 sentence1 相似度
sen_sen1 = util.cos_sim(bge_emb, bge_emb1)
print("{} 和 {} 相似度: {}".format(sentence, sentence1, sen_sen1.item()))

# sentence 和 sentence2 相似度
sen_sen2 = util.cos_sim(bge_emb, bge_emb2)
print("{} 和 {} 相似度: {}".format(sentence, sentence2, sen_sen2.item()))

**************************************************
                 bge-base-zh-v1
**************************************************
带宽超限 和 带宽使用超过阈值 相似度: 0.83314448595047
带宽超限 和 流量超阈值 相似度: 0.7644125819206238


In [1]:
def jaccard_similarity(str1, str2):
    list1 = str1.split()
    list2 = str2.split()
    intersection_list =set(list1) & set(list2)
    union_list =set(list1) | set(list2)
    similarity = len(intersection_list) / len(union_list)
    return similarity

In [6]:
sentence1 = "[ 数据 ],[ 描述 ]<<\d>,<\d>,<\d>.<\d>.<\d>.<\d>>DRTPReceive() failed:"
sentence2 = "[ 数据 ],[ 描述 ]<<\d>,<\d>,<\d>.<\d>.<\d>.<\d>>DRTPReceive() failed:"

In [7]:
print("{}   {}  Jaccard相似度:{}".format(sentence1, sentence2, jaccard_similarity(sentence1,sentence2)))
# print("{}   {}  Jaccard相似度:{}".format(sentence, sentence2, jaccard_similarity(sentence,sentence2)))

[ 数据 ],[ 描述 ]<<\d>,<\d>,<\d>.<\d>.<\d>.<\d>>DRTPReceive() failed:   [ 数据 ],[ 描述 ]<<\d>,<\d>,<\d>.<\d>.<\d>.<\d>>DRTPReceive() failed:  Jaccard相似度:1.0


In [17]:
l = [1,2,3]
r = l.pop()
r

3

In [1]:
import pandas as pd
nb_path = "/Users/zyl/Desktop/宁波银行告警标签库.csv"
dy_path = "/Users/zyl/Desktop/东亚银行模版告警.csv"
df = pd.read_csv(nb_path)
df

,告警内容,层级,标签
0,11.99.76.229 Red Host memory usage,服务器性能层,内存使用率高
1,"设备SHM201_XuHui_AS_01Ping不通,请关注!!",网络层,Ping不通
2,"设备SHM201_XuHui_AS_01Ping成功率为:0%,低于阀值~50,请关注!!",网络层,Ping成功率低
3,（10.21.254.243）线路【上海徐汇支行办公线路[运营商线路]】Rping10.21...,网络层,线路中断
4,"20.99.36.212:8081,cpss-settle-service 应用状态DOWN",应用层,应用状态DOWN
...,...,...,...
506,source=NB004_DWDM_07_电信、移动/NB004_DWDM_07（E9612...,系统设备,通信质量告警
507,宁波银行江北数据中心/2#楼/3F/3-302 3F-动力变-A/12Ca3-22 /12C...,系统设备,通讯异常
508,"偏离ntp时钟超过60秒,最新值1.1 Mmsec,【NTP:系统管理员处理】",服务器性能层,NTP时钟偏离
509,11.99.107.225 Red Snap Shot Monitor,服务器性能层,Snap Shot Monitor状态异常


In [4]:
new = df.drop_duplicates(subset=['层级', '标签']).reset_index(drop=True)
new.to_csv("/Users/zyl/Desktop/宁波银行告警标签库-去重.csv", index=False)

In [5]:
from pprint import pprint
label_map = df['gpt_m3e_label(修改)'].value_counts().to_dict()
print(len(label_map))

KeyError: 'gpt_m3e_label(修改)'

In [34]:
pprint(label_map)

85
{'BGP邻接关系异常': 4,
 'CPU使用率异常': 13,
 'IO服务时间异常': 1,
 'IP SLA状态变更': 2,
 'Ping超时': 5,
 'UPS电源异常': 5,
 'Zabbix 队列延迟': 2,
 'agent服务不可用': 3,
 'pool成员不可用': 6,
 'swap使用率高': 2,
 '丢包率高': 2,
 '主机名变更': 5,
 '作业状态异常': 157,
 '内存不足': 1,
 '内存使用率高': 6,
 '内管状态异常': 1,
 '分区状态异常': 1,
 '前置服务超时': 1,
 '前置机服务异常': 1,
 '前置机连通性异常': 1,
 '响应时间超阀值': 1,
 '备份失败': 15,
 '备机丢失': 1,
 '存储设备未挂载': 1,
 '对象存储服务进程异常': 1,
 '市电停电报警': 5,
 '带宽使用超过阈值': 36,
 '应用状态异常': 38,
 '开关断开报警': 2,
 '批处理告警': 20,
 '批量作业执行出错': 2,
 '接口丢包率超阈值': 10,
 '接口状态异常': 132,
 '数据不可访问': 1,
 '数据冗余降级': 1,
 '数据准备异常': 1,
 '数据库不可用': 1,
 '数据库发生重启': 1,
 '数据库状态异常': 16,
 '数据库连接失败': 1,
 '数据库错误': 8,
 '数据采集器负载高': 2,
 '文件更改': 1,
 '文件系统使用率高': 3,
 '日志异常告警': 2,
 '服务器心跳丢失': 1,
 '服务器监控状态变更': 3,
 '服务器通信断线': 3,
 '服务状态异常': 5,
 '水冷空调报警': 2,
 '流量超出阈值': 42,
 '消息队列异常': 2,
 '渠道状态异常': 1,
 '温度过高报警': 15,
 '漏水告警': 6,
 '物理磁盘异常告警': 2,
 '用户配置变更': 2,
 '电源状态异常': 5,
 '硬件传感器告警': 3,
 '硬件故障': 1,
 '磁盘响应时间长': 1,
 '磁盘空间使用率低': 4,
 '空调关机报警': 6,
 '端口异常告警': 2,
 '精密空调通信中断': 2,
 '系统交易错误': 6,
 '系统批处理告警': 25,


In [6]:
l = {'BGP邻接关系异常': 4,
 'CPU使用率异常': 13,
 'IO服务时间异常': 1,
 'IP SLA状态变更': 2,
 'Ping超时': 5,
 'UPS电源异常': 5,
 'Zabbix 队列延迟': 2,
 'agent服务不可用': 3,
 'pool成员不可用': 6,
 'swap使用率高': 2,
 '丢包率高': 2,
 '主机名变更': 5,
 '作业状态异常': 157,
 '内存不足': 1,
 '内存使用率高': 6,
 '内管状态异常': 1,
 '分区状态异常': 1,
 '前置服务超时': 1,
 '前置机服务异常': 1,
 '前置机连通性异常': 1,
 '响应时间超阀值': 1,
 '备份失败': 15,
 '备机丢失': 1,
 '存储设备未挂载': 1,
 '对象存储服务进程异常': 1,
 '市电停电报警': 5,
 '带宽使用超过阈值': 36,
 '应用状态异常': 38,
 '开关断开报警': 2,
 '批处理告警': 20,
 '批量作业执行出错': 2,
 '接口丢包率超阈值': 10,
 '接口状态异常': 132,
 '数据不可访问': 1,
 '数据冗余降级': 1,
 '数据准备异常': 1,
 '数据库不可用': 1,
 '数据库发生重启': 1,
 '数据库状态异常': 16,
 '数据库连接失败': 1,
 '数据库错误': 8,
 '数据采集器负载高': 2,
 '文件更改': 1,
 '文件系统使用率高': 3,
 '日志异常告警': 2,
 '服务器心跳丢失': 1,
 '服务器监控状态变更': 3,
 '服务器通信断线': 3,
 '服务状态异常': 5,
 '水冷空调报警': 2,
 '流量超出阈值': 42,
 '消息队列异常': 2,
 '渠道状态异常': 1,
 '温度过高报警': 15,
 '漏水告警': 6,
 '物理磁盘异常告警': 2,
 '用户配置变更': 2,
 '电源状态异常': 5,
 '硬件传感器告警': 3,
 '硬件故障': 1,
 '磁盘响应时间长': 1,
 '磁盘空间使用率低': 4,
 '空调关机报警': 6,
 '端口异常告警': 2,
 '精密空调通信中断': 2,
 '系统交易错误': 6,
 '系统批处理告警': 25,
 '系统服务异常': 3,
 '系统状态报警': 17,
 '网卡状态down': 1,
 '网络线路异常': 9,
 '节点维护': 3,
 '表空间使用率高': 5,
 '许可证过期告警': 2,
 '设备状态异常': 16,
 '设备通讯异常': 8,
 '运行出错': 1,
 '进程个数异常': 11,
 '进程负载高': 1,
 '连接状态异常': 6,
 '通讯异常': 16,
 '通道湿度异常': 2,
 '通道状态异常': 1,
 '配置变更': 12,
 '集群仲裁服务异常': 1}
l.keys()

dict_keys(['BGP邻接关系异常', 'CPU使用率异常', 'IO服务时间异常', 'IP SLA状态变更', 'Ping超时', 'UPS电源异常', 'Zabbix 队列延迟', 'agent服务不可用', 'pool成员不可用', 'swap使用率高', '丢包率高', '主机名变更', '作业状态异常', '内存不足', '内存使用率高', '内管状态异常', '分区状态异常', '前置服务超时', '前置机服务异常', '前置机连通性异常', '响应时间超阀值', '备份失败', '备机丢失', '存储设备未挂载', '对象存储服务进程异常', '市电停电报警', '带宽使用超过阈值', '应用状态异常', '开关断开报警', '批处理告警', '批量作业执行出错', '接口丢包率超阈值', '接口状态异常', '数据不可访问', '数据冗余降级', '数据准备异常', '数据库不可用', '数据库发生重启', '数据库状态异常', '数据库连接失败', '数据库错误', '数据采集器负载高', '文件更改', '文件系统使用率高', '日志异常告警', '服务器心跳丢失', '服务器监控状态变更', '服务器通信断线', '服务状态异常', '水冷空调报警', '流量超出阈值', '消息队列异常', '渠道状态异常', '温度过高报警', '漏水告警', '物理磁盘异常告警', '用户配置变更', '电源状态异常', '硬件传感器告警', '硬件故障', '磁盘响应时间长', '磁盘空间使用率低', '空调关机报警', '端口异常告警', '精密空调通信中断', '系统交易错误', '系统批处理告警', '系统服务异常', '系统状态报警', '网卡状态down', '网络线路异常', '节点维护', '表空间使用率高', '许可证过期告警', '设备状态异常', '设备通讯异常', '运行出错', '进程个数异常', '进程负载高', '连接状态异常', '通讯异常', '通道湿度异常', '通道状态异常', '配置变更', '集群仲裁服务异常'])